### Some preliminar things:
Let $\Omega \subset\mathbb{R}^2$ then we difine the following:

$$ \text{curl}: C^\infty(\Omega)^2 \ni \begin{pmatrix} u_1 \\u_2 \end{pmatrix}\to \partial_1 u_2 - \partial_2 u_1 \in C^\infty(\Omega) $$
$$ \text{Curl}: C^\infty(\Omega)^{2\times 2 } \ni \begin{pmatrix} u_{11} &u_{12}\\ u_{21} & u_{22}\\ \end{pmatrix}
\to   
\begin{pmatrix} \partial_1u_{12}-\partial_2 u_{11} \\ \partial_1u_{22}-\partial_2 u_{21} \\ \end{pmatrix} \in C^\infty(\Omega)^2 $$
$$ \text{rot}: C^\infty(\Omega)^1 \ni f \to \begin{pmatrix} \partial_2 f \\ -\partial_1 f  \end{pmatrix} \in C^\infty(\Omega)^2 $$
$$ \text{Rot}: C^\infty(\Omega)^{2 } \ni 
\begin{pmatrix} u_{1}\\ u_{2}  \end{pmatrix}
\to   
\begin{pmatrix} \partial_2u_{1} & -\partial_1 u_{1} \\ \partial_2 u_{2} &-\partial_1 u_{2} \\ \end{pmatrix}
\in C^\infty(\Omega)^{2\times 2} $$

It is easy to verify that :
$$
\int_{\Omega} curl ( u ) f = \int_{\Omega} u \cdot rot (f) + \int_{\partial \Omega} (u \cdot t) f
$$

$$
\int_{\Omega} Curl ( \sigma ) \cdot u = \int_{\Omega} \sigma \cdot Rot (u) + \int_{\partial \Omega} (\sigma t )\cdot u
$$

$$
\int_\Omega Inc(\sigma) \phi = \int_\Omega \sigma Rot \, rot(\phi) 
$$
Splitting onto the triangulation becomes:
$$
\sum_T \int_T \sigma Rot \, rot(\phi) 
$$
$$
\sum_T \int_T Curl(\sigma) \cdot rot(\phi) - \int_{\partial T} \sigma_t \cdot rot(\phi)
$$
We now split in tangential and normal componen the last integral, sinco the derivative is allowed only on the faces of the triangle and not accross:
$$
\sum_T \int_T Curl(\sigma) \cdot rot(\phi) - \int_{\partial T} \sigma_{tt} rot(\phi)\cdot t + \sigma_{nt} rot(\phi)\cdot n
$$

Since $\phi $ is smooth , $\sigma_{tt}$ is continuous accross the interfaces and $rot( \phi )\times t = n \cdot \nabla phi $ that cancels because the normal changes sign... then...

$$
\sum_T \int_T Curl(\sigma) \cdot rot(\phi) - \int_{\partial T}  \sigma_{nt} rot(\phi)\cdot n
$$
 Similarly $rot(\phi)\cdot n = - t \cdot \nabla \phi$ 

$$
\sum_T \int_T Curl(\sigma) \cdot rot(\phi) + \int_{\partial T} \sigma_{nt}  t \cdot \nabla \phi
$$

$$
\sum_T \int_T curl \, Curl(\sigma) \phi-\int_{\partial T} Curl(\sigma) \cdot t \phi + \int_{\partial T} \sigma_{nt}  t \cdot \nabla \phi
$$

$$
\sum_T \int_T inc (\sigma) \phi-\int_{\partial T} Curl(\sigma) \cdot t \phi - \int_{\partial T}  t \cdot \nabla (\sigma_{nt})  \phi
+ \int_{\partial T}t \cdot \nabla   (\sigma_{nt}  \phi)
$$
that is:

$$
\sum_T \int_T inc (\sigma) \phi-\int_{\partial T} Curl(\sigma) \cdot t \phi - \int_{\partial T}  t \cdot \nabla (\sigma_{nt})  \phi
+   \left. (\sigma_{nt}  \phi) \right \vert_{V_1}^{V_2}
$$
Where $\left.\left.\right.\right \vert_{V_1}^{V_2}$ Is hust the two vertices evaluation

$$


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square

In [2]:
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))

def Cof(u):
    v = CF( (u[3], u[2], u[1] ,u[0]) , dims=(2,2))
    return v

def Inc(u):
    return u[3].Diff(x).Diff(x) + u[0].Diff(y).Diff(y) - 2*u[1].Diff(x).Diff(y)


We need to imlement the following with $\sigma$ in $Reg^k$ and $\phi$ in lagrangian of order $k+1$ since the integrals over the Edges have to be well defined and the point evaluation is needed, therefore if $k=0$ then we need some continuity on the vertices!
$$
\sum_T \int_T inc (\sigma) \phi\quad-\quad\sum_E\int_{E} Curl(\sigma) \cdot t \phi \quad-\quad \left.\sum_E\int_{E}  n^T (\frac{\partial \sigma}{\partial t}) t  \phi
\quad+\quad   \sum_{"V"} [\sigma_{nt}  \phi] \right\vert_{V_1}^{V_2}
$$

In [3]:
def RunP(Maxh,Order):
    # Example of dirichlet boundary condition for the inc in 2D
    mesh = Mesh(unit_square.GenerateMesh(maxh=Maxh))
    # we set only dirichlet BND
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet=".*")
    fesH1 =  H1(mesh, order=Order+1, dirichlet= ".*")

    with TaskManager():

        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # create a continuous functions for our test
        zer = 1 #x*y
        #peak1 = zer*exp(-25*( (x-0.5)**2))
        #peak2 = zer*exp(-25*( (x-0.5)**2 + (y-0.5)**2))
        #peak3 = zer*exp(-25*(  (y-0.5)**2))
        peak1 = x**4 + y**4
        peak2 = (x**2)*(y**2)
        peak3 = x**4 + y**4
        PEAK = CF ( (peak1, peak2, peak2 ,peak3), dims=(2,2))
        # exct differentiation needed for the boundaries and to check the convergence
        IncPeak = CF( Inc(PEAK), dims=(1,1) ) 
        
        # grid functions 
        gfG = GridFunction(fesHCurlCurl) 
        gfG.Set ( CF ( PEAK, dims=(2,2)), bonus_intorder=15, dual=True)

        InterpIncgfG = GridFunction(fesH1)
        InterpIncgfG.Set (  IncPeak, bonus_intorder=15, dual=True)

        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################
        u,v = fesH1.TnT()

        # Mass matrix
        a = BilinearForm(fesH1)
        a += InnerProduct(u,v)*dx 

        # linear form induced by the metric gfG
        f = LinearForm(fesH1)

        # triangle inc part
        f += (gfG.Operator("inc"))*v*dx

        # edge curl part
        t = specialcf.tangential(2)
        f += -v*InnerProduct(curl(gfG),t)*dx(element_vb=BND)
        
        # edge tangential gradient part
        n = specialcf.normal(2)
        tdergfG = grad(gfG)[t,:]
        print(tdergfG.dims)
        f += v*InnerProduct( t,tdergfG.Reshape((2,2))*n)*dx(element_vb=BND) 

        # vertex elements   
        Vt = specialcf.VertexTangentialVectors(2)
            # I need the normals to be out from the triangle:
        Vn0 = CF( (CF((0,-1,1,0), dims= (2,2))*Vt[:,0])  )
        Vn1 = CF( (CF((0,1,-1,0), dims= (2,2))*Vt[:,1])  ) 

        f += v*InnerProduct( Vt[:,0],gfG*Vn0)*dx(element_vb=BBND) 
        f += v*InnerProduct( Vt[:,1],gfG*Vn1)*dx(element_vb=BBND) 

        
        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################
        #
        # In this part we need to implement the homogeization process
        # for our system: u_0F= A_FF ^-1 (f_F - [A u_D]_F )  ;
        # to rimplify we use an intermediate step | r = f_F - [A u_D]_F
        #                                         | u_0F= A_FF ^-1 r  


        # Assemblation of the 2 forms
        a.Assemble()
        f.Assemble()

        # Set the Dirichlet boundary part
        gfCurlCurlG = GridFunction(fesH1)
        gfCurlCurlG.Set(IncPeak, BND)
        #Draw(gfCurlCurlG)
        
        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlCurlG.vec
        gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesH1.FreeDofs(),inverse="sparsecholesky") * r

        print("Real inc of PEAK function")
        Draw (IncPeak, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False)
        print("my inc of PEAK function")
        Draw (gfCurlCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        print("Error  when order is "+str(Order)+"-->"+str(norm(IncPeak-gfCurlCurlG,mesh)) )
        Draw (IncPeak-gfCurlCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=True)
        
        
        return norm(IncPeak-gfCurlCurlG,mesh)




In [4]:
Maxh = 0.2
Order = 2
with TaskManager():
    RunP(Maxh, Order)


0: 4

Real inc of PEAK function


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

my inc of PEAK function


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

Error  when order is 2-->6.04811126598079e-11


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…